### Make Predictions with k-nearest neighbors on the Iris Flowers Dataset

In [50]:
# Import library for the project
from csv import reader
from math import sqrt
import random

Open and load the csv

In [16]:
# Load a CSV file
def load_csv(filename):
    dataset = list()
    with open(filename, 'r') as file:
        csv_reader = reader(file)
        for row in csv_reader:
            if not row:
                continue
            dataset.append(row)
    return dataset

I split in 2 part the dataset (randomly), the first part is the training data and the second part will be use for the validation model

In [79]:
# split in 2 the dataset 
def splitcsv(dataset):
    validation_data = []
    trainning_data = []
    for i in range(0,len(dataset)):
        rd = random.choice([True, False])
        if rd:
            trainning_data.append(dataset[i]) 
        else:
            validation_data.append(dataset[i])
    return trainning_data, validation_data

    

Define function from scratch :

-Convert string column to float

-Convert string column to integer

-Find the min and max values for each column

-Rescale dataset columns to the range 0-1 (normalization)

-Calculate the Euclidean distance between two vectors

-Locate the most similar neighbors

In [18]:
def convert_str_column_to_float(dataset, column):
    for row in dataset:
        row[column] = float(row[column].strip())

def str_column_to_int(dataset, column):
    class_values = [row[column] for row in dataset]
    unique = set(class_values)
    lookup = dict()
    for i, value in enumerate(unique):
        lookup[value] = i
        print('[%s] => %d' % (value, i))
    for row in dataset:
        row[column] = lookup[row[column]]
    return lookup

def dataset_minmax(dataset):
    minmax = list()
    for i in range(len(dataset[0])):
        col_values = [row[i] for row in dataset]
        value_min = min(col_values)
        value_max = max(col_values)
        minmax.append([value_min, value_max])
    return minmax

def normalize_dataset(dataset, minmax):
    for row in dataset:
        for i in range(len(row)):
            row[i] = (row[i] - minmax[i][0]) / (minmax[i][1] - minmax[i][0])

def distance_euclid(row1, row2):
    distance = 0.0
    for i in range(len(row1)-1):
        distance += (row1[i] - row2[i])**2
    return sqrt(distance)

def find_neighbors(train, test_row, num_neighbors):
    distances = list()
    for train_row in train:
        dist = distance_euclid(test_row, train_row)
        distances.append((train_row, dist))
    distances.sort(key=lambda tup: tup[1])
    neighbors = list()
    for i in range(num_neighbors):
        neighbors.append(distances[i][0])
    return neighbors

Prediction
Make a prediction with neighbors

In [19]:
def prediction(train, test_row, num_neighbors):
    neighbors = find_neighbors(train, test_row, num_neighbors)
    output_values = [row[-1] for row in neighbors]
    prediction = max(set(output_values), key=output_values.count)
    return prediction

Make a prediction with KNN on Iris Dataset prediction --> row = 5.7,2.9,4.2,1.3

In [98]:
filename = 'iris.csv'
dataset = load_csv(filename)
dataset_train, dataset_validation = splitcsv(dataset)
for i in range(len(dataset_train[0])-1):
	convert_str_column_to_float(dataset_train, i)
for i in range(len(dataset_validation[0])-1):
	convert_str_column_to_float(dataset_validation, i)
# convert class column to integers
str_column_to_int(dataset_train, len(dataset_train[0])-1)
str_column_to_int(dataset_validation, len(dataset_validation[0])-1)
# define model parameter
num_neighbors = 5

# predict the label
for j in range(0,len(dataset_validation)):
	label = 0
	label = prediction(dataset_train, dataset_validation[j][0:3], num_neighbors)
	print('Data=%s, Predicted: %s' % (dataset_validation[j], label))

[Iris-virginica] => 0
[Iris-setosa] => 1
[Iris-versicolor] => 2
[Iris-virginica] => 0
[Iris-setosa] => 1
[Iris-versicolor] => 2
Data=[5.1, 3.5, 1.4, 0.2, 1], Predicted: 1
Data=[4.9, 3.0, 1.4, 0.2, 1], Predicted: 1
Data=[4.7, 3.2, 1.3, 0.2, 1], Predicted: 1
Data=[4.6, 3.1, 1.5, 0.2, 1], Predicted: 1
Data=[5.4, 3.9, 1.7, 0.4, 1], Predicted: 1
Data=[4.6, 3.4, 1.4, 0.3, 1], Predicted: 1
Data=[4.4, 2.9, 1.4, 0.2, 1], Predicted: 1
Data=[4.9, 3.1, 1.5, 0.1, 1], Predicted: 1
Data=[5.1, 3.5, 1.4, 0.3, 1], Predicted: 1
Data=[5.1, 3.8, 1.5, 0.3, 1], Predicted: 1
Data=[4.6, 3.6, 1.0, 0.2, 1], Predicted: 1
Data=[5.0, 3.0, 1.6, 0.2, 1], Predicted: 1
Data=[5.2, 3.5, 1.5, 0.2, 1], Predicted: 1
Data=[4.7, 3.2, 1.6, 0.2, 1], Predicted: 1
Data=[4.8, 3.1, 1.6, 0.2, 1], Predicted: 1
Data=[5.2, 4.1, 1.5, 0.1, 1], Predicted: 1
Data=[5.5, 4.2, 1.4, 0.2, 1], Predicted: 1
Data=[4.9, 3.1, 1.5, 0.1, 1], Predicted: 1
Data=[5.5, 3.5, 1.3, 0.2, 1], Predicted: 1
Data=[5.0, 3.5, 1.3, 0.3, 1], Predicted: 1
Data=[4.5, 2